In [1]:
import pandas as pd
import glob
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_asd_data_with_pandas(data_path: str = "/content/drive/MyDrive/Antibody_Chekalin_Denis/asd") -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        # print(f"Загружаем файл: {os.path.basename(file_path)}")
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


In [ ]:
# Удаляем дубликаты !!!
# полные дубликаты удаляем, (где метадаты разные но остальное одинаковое, объединить метадаты)
agab_df_temp = agab_df.drop_duplicates(subset=['light_sequence', 'heavy_sequence'])
print(f'agab_df без дубликатов = {len(agab_df_temp)}')

paired_agab_df = agab_df[
    agab_df['light_sequence'].notna()
    & agab_df['heavy_sequence'].notna()
    & (agab_df['light_sequence'] != '')
    & (agab_df['heavy_sequence'] != '')
    & (agab_df['nanobody'] == False)
]
print(f'agab_df после фильтра = {len(paired_agab_df)}')

agab_df без дубликатов = 865280
agab_df после фильтра = 713227


(865280, 11)

In [ ]:
paired_agab_df

Сделай .fasta в таком формате:
>ABC123_H
EVQLV...

>ABC123_L
DIQMT...
Где ABC123 — это ID пары.